In [1]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome

def get_tweet_data(tweet):
    username = tweet.find_element_by_xpath('.//span').text
    handle = tweet.find_element_by_xpath ('.//span[ contains(text(), "@")]').text
    try:
        postdate = tweet.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    tweet_content = comment + responding
    replies = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweets = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    likes = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
    tweet = (username, handle, postdate, tweet_content, replies, retweets, likes)
    return tweet

driver = Chrome()
driver.get('https://www.twitter.com/login')
driver.maximize_window()
sleep(5)
username = driver.find_element_by_xpath('//input[@name="username"]')
username.send_keys('Your Username')
username.send_keys(Keys.RETURN)

my_password = getpass()
password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

sleep(3)
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input. send_keys ('#your_hashtag')
search_input.send_keys(Keys.RETURN) 

driver.find_element_by_link_text('Latest').click()

tweet_data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
sleep(2)
while scrolling: 
    tweets = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    
    for i in tweets[-15: ]: 
        tweet = get_tweet_data(i)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id) 
                tweet_data.append(tweet)
    
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document. body. scrollHeight);')
        sleep(2)
        current_position = driver.execute_script("return window.pageYOffset;")
        if last_position == current_position:
            scroll_attempt += 1
            
            if scroll_attempt >= 3:
                scrolling = False
                break
            else: 
                sleep(2)
        else: 
            last_position = current_position
            break
    

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [2]:
len(tweet_data)

31

In [3]:
with open ('tweets.csv', 'w', newline='', encoding='utf-8') as f:
    table_headings = ['UserName', 'Handle', 'PostDate', 'Content', 'Replies', 'Retweets', 'Likes']
    writer = csv.writer(f)
    writer.writerow(table_headings)
    writer.writerows(tweet_data)